In [1]:
import re
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
# 不用儲存article_id, 轉成list
def training_txt_to_list(path):
    with open(path, 'r') as f:
        txt = str(f.read())
    txt_list = txt.split('\n')
    text_label_list = list()
    tmp = list()
    for line in txt_list:
        if line == '--------------------':
            text_label_list.append(tmp)
            tmp = list()
            continue
        if line == '':
            continue
        tmp.append(line)
    x = list()
    y = list()
    for text_label in text_label_list:
        text = text_label[0]
        label = text_label[2:]
        label_list = ['O' for i in range(len(text))]
        for i in label:
            entity = i.split('\t')
            if int(entity[1]) > int(entity[2]):
                continue
            b = int(entity[1])
            label_list[b] = 'B-{}'.format(entity[-1])
            for j in range(int(entity[1])+1, int(entity[2])):
                label_list[j] = 'I-{}'.format(entity[-1])
        for i, j in enumerate(text):
            if j == '，' or j == '。' or j == '？':
                label_list[i] = j
                
        text_list = re.split('\uff0c|\u3002|\uff1f', text)
        for sentence in text_list:
            x.append([i for i in sentence])
        x = x[:-1]
            
        sentence = list()
        for i in label_list:
            if i == '，' or i == '。' or i == '？':
                y.append(sentence)
                sentence = list()
            else:
                sentence.append(i)
                
    return [x, y]

In [3]:
x1, y1 = training_txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage1/SampleData_deid.txt')
x2, y2 = training_txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage2/train_1_update.txt')
x4, y4 = training_txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage4/train_2.txt')

In [4]:
x = x1 + x2 + x4
y = y1 + y2 + y4
print(len(x))
print(len(y))

76162
76162


In [5]:
df = pd.DataFrame({'x': x, 'y': y})

In [6]:
df['x'] = df['x'].apply(lambda row : str(row))
df['y'] = df['y'].apply(lambda row : str(row))
df = df.drop_duplicates()
df['x'] = df['x'].apply(lambda row : eval(row))
df['y'] = df['y'].apply(lambda row : eval(row))
df

,x,y
0,"[醫, 師, ：, 你, 有, 做, 超, 音, 波, 嘛]","[O, O, O, O, O, O, O, O, O, O]"
1,"[那, 我, 們, 來, 看, 報, 告]","[O, O, O, O, O, O, O]"
2,"[有, 些, 部, 分, 有, 紅, 字, 耶]","[O, O, O, O, O, O, O, O]"
3,"[民, 眾, ：, 紅, 字, 是, 甚, 麼, 意, 思]","[O, O, O, O, O, O, O, O, O, O]"
4,"[醫, 師, ：, 就, 是, 肝, 功, 能, 有, 比, 較, 高]","[O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...
75713,"[醫, 師, ：, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID]"
75714,"[醫, 師, ：, 6, 6, 6, 0, 8, 9, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ..."
75716,"[護, 理, 師, ：, 6, 6, 6, 0, 9, 8]","[O, O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID]"
75717,"[醫, 師, ：, 0, 9, 8, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ..."


In [7]:
df = df.reset_index(drop=True)
df

,x,y
0,"[醫, 師, ：, 你, 有, 做, 超, 音, 波, 嘛]","[O, O, O, O, O, O, O, O, O, O]"
1,"[那, 我, 們, 來, 看, 報, 告]","[O, O, O, O, O, O, O]"
2,"[有, 些, 部, 分, 有, 紅, 字, 耶]","[O, O, O, O, O, O, O, O]"
3,"[民, 眾, ：, 紅, 字, 是, 甚, 麼, 意, 思]","[O, O, O, O, O, O, O, O, O, O]"
4,"[醫, 師, ：, 就, 是, 肝, 功, 能, 有, 比, 較, 高]","[O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...
31516,"[醫, 師, ：, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID]"
31517,"[醫, 師, ：, 6, 6, 6, 0, 8, 9, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ..."
31518,"[護, 理, 師, ：, 6, 6, 6, 0, 9, 8]","[O, O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID]"
31519,"[醫, 師, ：, 0, 9, 8, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ..."


In [8]:
def find_contain_entity_rows(labels):
    dic_temp = {}
    dummy = 0
    for label in labels:
        dic_temp[label] = dummy
    #if len(dic_temp) > 1:
    #    return True
    if 'B-clinical_event' in dic_temp:
        return True
    else:
        return False
df['flag'] = df['y'].apply(lambda l:find_contain_entity_rows(l))

In [9]:
df

,x,y,flag
0,"[醫, 師, ：, 你, 有, 做, 超, 音, 波, 嘛]","[O, O, O, O, O, O, O, O, O, O]",False
1,"[那, 我, 們, 來, 看, 報, 告]","[O, O, O, O, O, O, O]",False
2,"[有, 些, 部, 分, 有, 紅, 字, 耶]","[O, O, O, O, O, O, O, O]",False
3,"[民, 眾, ：, 紅, 字, 是, 甚, 麼, 意, 思]","[O, O, O, O, O, O, O, O, O, O]",False
4,"[醫, 師, ：, 就, 是, 肝, 功, 能, 有, 比, 較, 高]","[O, O, O, O, O, O, O, O, O, O, O, O]",False
...,...,...,...
31516,"[醫, 師, ：, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID]",False
31517,"[醫, 師, ：, 6, 6, 6, 0, 8, 9, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ...",False
31518,"[護, 理, 師, ：, 6, 6, 6, 0, 9, 8]","[O, O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID]",False
31519,"[醫, 師, ：, 0, 9, 8, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ...",False


In [10]:
df_entity = df[df['flag'] == True]

In [12]:
df_entity = df_entity.reset_index(drop=True)

In [13]:
df_entity

,x,y,flag
0,"[醫, 師, ：, 那, 時, 候, 在, 做, H, 1, N, 1]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c...",True
1,"[那, 時, 候, 還, 沒, 有, 那, 個, H, 1, N, 1]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c...",True
2,"[民, 眾, ：, 然, 後, 下, 次, 就, H, 1, N, 1]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c...",True
3,"[S, A, R, S]","[B-clinical_event, I-clinical_event, I-clinica...",True
4,"[就, 像, 大, 家, 在, 弄, 這, 個, S, A, R, S]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c...",True


In [14]:
clinical_df = pd.read_csv('../dictionary/online/clinical_event.csv')

In [32]:
aug_df = pd.DataFrame([],columns=['x','y'])
aug_df

,x,y


In [33]:
for index, row in df_entity.iterrows():
    entity_start_pos = 0
    entity_end_pos = -1
    start_flag = False
    for i in range(len(row['y'])):
        if row['y'][i] == 'B-clinical_event':
            entity_start_pos = i
            start_flag = True
        elif row['y'][i] == 'O' and start_flag == True:
            entity_end_pos = i
            break
        else:
            continue
    for entity in clinical_df['entity_text']:
        sentence = []
        labels = []
        count = 0
        for i in range(entity_start_pos):
            sentence.append(row['x'][i])
            labels.append('O')
        sentence.append(entity[0])
        labels.append('B-clinical_event')
        for i in range(1,len(entity)):
            sentence.append(entity[i])
            labels.append('I-clinical_event')
        if entity_end_pos != -1:
            for i in range(entity_end_pos,len(row['x'])):
                sentence.append(row['x'][i])
                labels.append('O')
        if len(sentence) != len(labels):
            print('Wrong')
        aug_df = aug_df.append({'x':sentence,'y':labels},ignore_index=True)
    

In [34]:
aug_df

,x,y
0,"[醫, 師, ：, 那, 時, 候, 在, 做, 腦, 炎]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c..."
1,"[醫, 師, ：, 那, 時, 候, 在, 做, 日, 本, 腦, 炎]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c..."
2,"[醫, 師, ：, 那, 時, 候, 在, 做, 小, 兒, 痲, 痺, 症]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c..."
3,"[醫, 師, ：, 那, 時, 候, 在, 做, 帕, 金, 森, 病]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c..."
4,"[醫, 師, ：, 那, 時, 候, 在, 做, 阿, 茲, 海, 默, 症]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c..."
...,...,...
3445,"[就, 像, 大, 家, 在, 弄, 這, 個, V, W, M]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c..."
3446,"[就, 像, 大, 家, 在, 弄, 這, 個, W, A, G, R]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c..."
3447,"[就, 像, 大, 家, 在, 弄, 這, 個, X, X, X]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c..."
3448,"[就, 像, 大, 家, 在, 弄, 這, 個, Y, Y]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c..."


In [35]:
aug_df = aug_df.sample(500)
aug_df = aug_df.reset_index(drop=True)
aug_df

,x,y
0,"[民, 眾, ：, 然, 後, 下, 次, 就, Q, 型, 流, 感]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c..."
1,"[D, H, F]","[B-clinical_event, I-clinical_event, I-clinica..."
2,"[那, 時, 候, 還, 沒, 有, 那, 個, 嚴, 重, 急, 性, 呼, 吸, 道, ...","[O, O, O, O, O, O, O, O, B-clinical_event, I-c..."
3,"[就, 像, 大, 家, 在, 弄, 這, 個, C, S, D]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c..."
4,"[那, 時, 候, 還, 沒, 有, 那, 個, 性, 偏, 好, 障, 礙]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c..."
...,...,...
495,"[醫, 師, ：, 那, 時, 候, 在, 做, T, C, S]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c..."
496,"[那, 時, 候, 還, 沒, 有, 那, 個, 陰, 虱]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c..."
497,"[醫, 師, ：, 那, 時, 候, 在, 做, 肝, 癌]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c..."
498,"[就, 像, 大, 家, 在, 弄, 這, 個, P, L, S]","[O, O, O, O, O, O, O, O, B-clinical_event, I-c..."


In [36]:
aug_df.to_csv('../datasets/clinical_event_augmentation.csv',index=False)

In [37]:
aug_df.loc[100,'x']

['民', '眾', '：', '然', '後', '下', '次', '就', 'F', 'T', 'D']

In [38]:
aug_df.loc[100,'y']

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-clinical_event',
 'I-clinical_event',
 'I-clinical_event']